In [ ]:
#Yelp 2013

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, PredefinedSplit


In [129]:
#load reviews
yelp = pd.read_csv('yelp_2013_train.csv')

#add review length to data framce
yelp['text length'] = yelp['text'].str.len()

#sample for testing purppses
#yelp = yelp.sample(frac=.5)

X = yelp['text']
y = yelp['stars']

In [11]:
#remove capitalization, stopwords, and punctuation
def process(text) : 
    #remove punctuation/captizalization
    text = str(text)
    text = text.lower()
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    #remove stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]


In [130]:

max_vocab = 10000

bow_transformer = CountVectorizer(analyzer=process, max_features=max_vocab).fit(X)


In [131]:
X = bow_transformer.transform(X)

In [132]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


print(y_train.shape)

(160934,)


In [133]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)

preds = nb.predict(X_test)


In [81]:
#SVM

def linSVM(parameters, X, T, fold) : 
    n_folds = 5
    svc = LinearSVC(multi_class='ovr')
    svc_cv = GridSearchCV(svc, parameters, cv=fold, scoring="f1_micro", refit=False)
    svc_cv.fit(X, T)
    scores = svc_cv.cv_results_['mean_test_score']
    params = svc_cv.cv_results_['params']
    print('scores:',scores)
    print('params:',params)
    return svc_cv.best_params_


#Predefined Split for CV
my_test_fold = []
for i in range(X_train.shape[0]):
    my_test_fold.append(-1)
for i in range(y_train.shape[0]):
    my_test_fold.append(0)

fold = PredefinedSplit(test_fold=my_test_fold)
CV_split_T = np.append(y_train, y_test)

import scipy.sparse as sp
CV_split_X = sp.vstack((X_train , X_test))  # NOT np.vstack


print(CV_split_T.shape)
print(CV_split_X.shape)


(114954,)
(114954, 20000)
114954


In [140]:
Cs = [0.0018, 0.0019, 0.002, 0.0021, 0.0022, 0.0023, 0.0024]
#Cs = [0.00175, 0.00177, 0.00179, 0.00180, 0.00182, 0.00184, 0.00186]
Cs = [0.001892, 0.001894, 0.001896, 0.001898, 0.00190, 0.00192, 0.00194]
Cs = [0.001890, 0.001891, 0.001892, 0.001893, 0.001894]

param_grid = {'C': Cs}

#bestParams = linSVM(param_grid, CV_split_X, CV_split_T, fold)


svc = LinearSVC(C = .001893, multi_class='ovr')
svc.fit(X_train, y_train)


preds = nb.predict(X_test)

In [141]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

print(confusion_matrix(y_test, preds))
print('\n')
print(classification_report(y_test, preds))



print(accuracy_score(y_test, preds))

[[ 3186  1340   329   218   187]
 [ 1218  2246  1889   556   274]
 [  796  1233  4341  3506   793]
 [  749   613  2775 13367  6587]
 [ 1077   226   505  6771 14191]]


             precision    recall  f1-score   support

          1       0.45      0.61      0.52      5260
          2       0.40      0.36      0.38      6183
          3       0.44      0.41      0.42     10669
          4       0.55      0.55      0.55     24091
          5       0.64      0.62      0.63     22770

avg / total       0.54      0.54      0.54     68973

0.5412407753758717
